# Unit 2 Exploring MCP Primitives: Tools, Resources, and Prompts

-----

## Introduction and Lesson Overview

Welcome back\! In the previous lesson, you learned what the **Model Context Protocol (MCP)** is, why it's important, and how to launch a basic MCP server in Python using the **FastMCP** class. You also explored the two main ways to run your server—using **standard input/output (stdio)** for local communication and **Server-Sent Events (SSE)** for networked scenarios. By now, you should feel comfortable starting an MCP server and understanding how it fits into the broader AI integration landscape.

In this lesson, we will build on that foundation by exploring how to define and expose your server’s capabilities. Specifically, you will learn how to add **tools**, **resources**, and **prompts** to your MCP server. These are the core building blocks that allow your server to provide useful functions, share data, and guide AI interactions. By the end of this lesson, you will know how to define each of these primitives and how to interact with them from a client. This will prepare you to create more powerful and interactive MCP servers.

-----

## Understanding MCP Server Primitives

To make your MCP server truly useful, you need to expose its capabilities in a way that clients (such as AI agents or other applications) can discover and use. MCP provides three main primitives for this purpose: **tools**, **resources**, and **prompts**.

  * **Tools** are executable functions that perform actions or computations. For example, a tool might add two numbers, fetch data from an API, or process a file. Tools are the “actions” your server can perform on request.
  * **Resources** are data entities that your server exposes. These can be static (like a greeting message or configuration) or dynamic (like user profiles or database records). Resources provide context or information that clients can read and use.
  * **Prompts** are reusable templates that guide AI interactions. They help structure the way an AI model asks questions, explains concepts, or interacts with users. Prompts can accept arguments and generate dynamic messages.

Understanding these primitives is essential because they are the standard way for AI models and clients to discover what your server can do. When you define tools, resources, and prompts, you make your server’s capabilities visible and accessible to the outside world.

-----

## Defining MCP Tools

Let’s start by defining a tool. In MCP, a tool is simply a Python function that you expose to clients using a decorator. The **FastMCP** library makes this easy with the `@mcp.tool()` decorator. When you decorate a function as a tool, it becomes discoverable and callable by clients.

Here is an example of a simple tool that adds two integers and returns the result:

```python
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="Provides tools, resources, and prompts"
)

# Define a tool that adds two integers and returns the result
@mcp.tool()
def add(a: int, b: int) -> int:
    """Return the sum of a and b."""
    return a + b
```

In this code, the `add` function takes two integer arguments, `a` and `b`, and returns their sum. The `@mcp.tool()` decorator registers this function as a tool with the MCP server. You can add as many tools as you like, each with its own logic and documentation.

When a client connects to your server, it can list all available tools and see their names, descriptions, and input parameters. This makes it easy for clients to discover what actions your server can perform.

-----

## Exposing MCP Resources

Next, let’s look at resources. Resources are pieces of data that your server exposes for clients to read. Each resource is identified by a unique URI (Uniform Resource Identifier), which acts like an address for that piece of data.

To define a resource in **FastMCP**, you use the `@mcp.resource()` decorator and provide a URI. Here is an example:

```python
# Define a resource that returns a static greeting string
@mcp.resource("resource://greeting", description="A simple greeting string returned as plain text.")
def greeting() -> str:
    """A simple greeting string returned as plain text."""
    return "Hello from My Server!"
```

Notice the `description` parameter in the decorator. While you might expect FastMCP to use the function’s docstring as the resource description, in practice, the docstring is not always picked up reliably due to current limitations in FastMCP’s docstring parsing. To ensure your resource has a clear, visible description when clients list available resources, you should always provide the `description` argument explicitly in the decorator.

In this example, the `greeting` function is registered as a resource with the URI `resource://greeting`. When a client requests this resource, the server returns the string "Hello from My Server\!". Resources can be static, like this greeting, or dynamic, returning different data based on input parameters or server state.

Resources are useful for sharing configuration, documentation, or any other data that clients might need to use as context for AI interactions.

-----

## Creating MCP Prompts

Prompts are a unique feature of MCP that allow you to define reusable templates for AI interactions. A prompt is a function that generates a message or instruction, often using input arguments to customize the output.

To define a prompt in **FastMCP**, use the `@mcp.prompt()` decorator. Here is an example:

```python
# Define a prompt template that generates a user message about a topic
@mcp.prompt()
def ask_about_topic(topic: str) -> str:
    """Generate a user message asking for an explanation of topic."""
    return f"Can you explain the concept of '{topic}' in simple terms?"
```

In this code, the `ask_about_topic` function takes a `topic` argument and returns a formatted string asking for an explanation of that topic. Prompts like this are helpful for guiding AI models to ask questions, explain concepts, or follow specific workflows.

When a client lists available prompts, it can see their names, descriptions, and required arguments. The client can then request a prompt with specific arguments to generate a customized message.

-----

## Connecting to the MCP Server with the MCP Client

To interact with your MCP server, you can use the `mcp` client interface. The client connects to the server using a transport, such as standard input/output (stdio). Here’s how you can launch the server and establish a connection from a client script:

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )
    # Start the stdio client and get the read/write streams for communication
    async with stdio_client(server_params) as (read, write):
        # Create a client session using the communication streams
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            # Client-Server interactions go here...

if __name__ == "__main__":
    asyncio.run(main())
```

This script prepares everything needed to launch your MCP server as a separate process (subprocess) and communicate with it using standard input and output streams (stdio). The process involves several key steps to establish a working connection between the client and the server.

The main steps are:

1.  **Define the server parameters**, including the command to run and any arguments (such as the server script filename).
2.  Use `stdio_client` to start the server process and obtain the low-level read and write streams for communication.
3.  Pass these streams to `ClientSession`, which manages the details of the MCP protocol, including sending requests and receiving responses.
4.  Call `await session.initialize()` to perform the initial handshake and set up the session.

By following these steps, you ensure that both the client and server are properly connected and ready to exchange MCP messages. Only after the session is initialized can you safely make requests such as listing tools, reading resources, or calling tools on the server.

-----

## Listing Available Tools, Resources, and Prompts

Once the client is connected, it can use the `list_tools`, `list_resources`, and `list_prompts` methods to dynamically discover all the capabilities the server exposes. These methods are important because they allow agents to explore available actions, data, and interaction templates at runtime, enabling flexible and adaptive use of the server’s features. The following code demonstrates how to list each type of primitive and print their names and descriptions, providing a clear overview of what the server offers.

```python
# List all tools the server provides
tools_response = await session.list_tools()
print("Available tools:")
for tool in tools_response.tools:
    print(f" - {tool.name}: {tool.description}")

# List all resources
resources_response = await session.list_resources()
print("\nAvailable resources:")
for res in resources_response.resources:
    print(f" - {res.uri}: {res.description}")

# Introspect prompts
prompts_response = await session.list_prompts()
print("\nAvailable prompts:")
for p in prompts_response.prompts:
    print(f" - {p.name}: {p.description}")
```

After running this code, you’ll see a summary of all tools, resources, and prompts that the server provides, making it easy to understand what actions and data are accessible:

```text
Available tools:
 - add: Return the sum of a and b.

Available resources:
 - resource://greeting: A simple greeting string returned as plain text.

Available prompts:
 - ask_about_topic: Generate a user message asking for an explanation of topic.
```

-----

## Calling a Tool from the Server

After discovering the available tools, you can invoke any tool by name using the `call_tool` method on the client session. This method allows you to execute server-side functions and retrieve their results by providing the tool’s name and the required arguments as a dictionary.

For example, to call the `add` tool (which takes two integers and returns their sum), you can use the following code:

```python
# Call the 'add' tool with arguments a=5 and b=3
result_tool = await session.call_tool("add", {"a": 5, "b": 3})
print(f"Result of add tool: {result_tool.content[0].text}")
```

When you run this code, the client sends a request to the server to execute the `add` tool with the specified arguments. The server processes the request and returns the result, which you can access from the first element of the returned content list using the `.text` attribute.

Example output:

```text
Result of add tool: 8
```

This mechanism allows clients to dynamically execute any tool exposed by the server, making it easy to integrate and automate server-side actions from your client applications.

-----

## Reading a Resource from the Server

Just as you can call tools by name and pass arguments, you can also access resources exposed by the server using their unique URIs. The example below shows how to read the `resource://greeting` resource and print its value. Since the server returns a list of resource instances, you access the first element’s `.text` property to get the actual greeting.

```python
# Fetch the greeting resource
result_resource = await session.read_resource("resource://greeting")
print("Greeting resource:", result_resource.contents[0].text)
```

When you run this code, the client retrieves the greeting resource and prints its content:

```text
Greeting resource: Hello from My Server!
```

-----

## Generating a Prompt with Arguments

Prompts can be used to generate dynamic messages by providing the required arguments. The following code demonstrates how to generate a prompt for a specific topic by calling the prompt with an argument and printing the resulting message. This is especially useful for guiding AI interactions or creating user-facing messages on the fly.

```python
# Fetch and display the formatted prompt for a topic
topic = "machine learning"
result_prompt = await session.get_prompt("ask_about_topic", {"topic": topic})
print(f"Prompt for topic '{topic}': {result_prompt.messages[0].content.text}")
```

Here, the client sends the topic "machine learning" to the prompt and prints the generated message:

```text
Prompt for topic 'machine learning': Can you explain the concept of 'machine learning' in simple terms?
```

This is the power of MCP: clients can dynamically explore and use whatever tools, resources, and prompts your server provides without needing to know the details in advance.

-----

## Summary and Next Steps

In this lesson, you learned how to define and expose the three main MCP server primitives: **tools**, **resources**, and **prompts**. **Tools** let your server perform actions or computations, **resources** provide data for clients to use as context, and **prompts** guide AI interactions with reusable templates. You also saw how a client can connect to your server, discover its capabilities, and interact with each primitive.

These building blocks are at the heart of the MCP framework. By mastering them, you can create servers that are both powerful and flexible, ready to integrate with AI agents and other applications. In the next section, you will get hands-on practice by defining your own tools, resources, and prompts, and by writing client code to interact with them. Happy coding\!

## Connecting Your First MCP Client

It’s time to put your knowledge into practice by building a simple client that connects to a basic MCP server.

The MCP server is set up for you and currently has no tools, resources, or prompts—just a name and description. Your task is to complete the main.py file so that it connects to the provided server (mcp_server.py) using stdio.

To complete this exercise, you should:

Set up the server parameters using StdioServerParameters so the client knows how to launch and connect to mcp_server.py.
Use the stdio_client to establish a connection to the server.
Create a client session using the ClientSession class and the communication streams.
Initialize the session with session.initialize() to perform the handshake and confirm the connection.
Print a message to show that the connection and initialization were successful.
This exercise will help you become comfortable with the basics of MCP client setup and session initialization.

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # TODO: Define server parameters for stdio connection to mcp_server.py

    # TODO: Establish stdio client connection
        # TODO: Create a client session using the communication streams
            # TODO: Initialize the connection
            # TODO: Print a success message when connected

if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection to mcp_server.py
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        # Create a client session using the communication streams
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            # Print a success message when connected
            print("Successfully connected to the MCP server and initialized the session!")

if __name__ == "__main__":
    asyncio.run(main())
```



## Discovering Server Tools in MCP

Now that your client can connect to the MCP server, let’s take the next step by adding a tool to the server and making sure your client can discover it.

Your task is to create a tool on the server that multiplies two numbers and returns the result. You’ll also update your client so that it lists all available tools and confirms that your new multiplication tool is visible.

To complete this exercise:

In mcp_server.py, define a function that multiplies two numbers and returns the product. Be sure to include a clear docstring for your function—MCP uses the function’s docstring as the tool’s description, making it discoverable and understandable to clients.
Register this function as a tool using the @mcp.tool() decorator so that it is discoverable by clients.
In main.py, after initializing the session, list all tools provided by the server.
Print the name and description of each tool to confirm that your multiplication tool appears in the list.
This will help you practice exposing server capabilities and verifying them from the client side.


```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool."
)


# TODO: Define a tool that multiplies two numbers and returns the result


if __name__ == "__main__":
    mcp.run(transport="stdio")

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # TODO: List all tools the server provides
            # TODO: Print the name and description of each tool

if __name__ == "__main__":
    asyncio.run(main())
```

Here are the updated `mcp_server.py` and `main.py` files to complete the exercise:

**`mcp_server.py`**

```python
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool."
)

# Define a tool that multiplies two numbers and returns the result
@mcp.tool()
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers and returns their product."""
    return a * b


if __name__ == "__main__":
    mcp.run(transport="stdio")
```

**`main.py`**

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()
            print("Successfully connected to the MCP server and initialized the session!\n")

            # List all tools the server provides
            tools_response = await session.list_tools()

            # Print the name and description of each tool
            print("Available tools:")
            if tools_response.tools:
                for tool in tools_response.tools:
                    print(f" - {tool.name}: {tool.description}")
            else:
                print("No tools found on the server.")


if __name__ == "__main__":
    asyncio.run(main())
```

**To run this example:**

1.  Save the first code block as `mcp_server.py`.
2.  Save the second code block as `main.py`.
3.  Run `python main.py` in your terminal.

You should see output similar to this, confirming that your `multiply` tool is discovered:

```
Successfully connected to the MCP server and initialized the session!

Available tools:
 - multiply: Multiplies two numbers and returns their product.
```

## Adding and Discovering Server Resources

Now, let’s expand your server’s capabilities by adding a resource and making sure your client can discover it.

Your task is to add a new resource to the server that returns a health status message. This resource should have a unique URI and a clear description. After updating the server, you’ll also update your client so it lists all available resources, showing their URIs and descriptions.

To complete this exercise:

In mcp_server.py, define a function that returns a health status message, such as "Server is healthy".
Register this function as a resource using the @mcp.resource() decorator, giving it a unique URI (for example, "resource://health") and a description. (Remember: always provide the description parameter explicitly, since FastMCP does not reliably extract the docstring for resources.)
In main.py, after listing the tools, add code to list all resources provided by the server.
Print each resource’s URI and description to confirm your new health resource is visible.
This will help you practice exposing server data and verifying it from the client side, making your MCP server more informative and useful.

```python
#mcp_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool and a health resource."
)


# Existing multiply tool
@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Return the product of a and b."""
    return a * b


# TODO: Define a resource that returns a health status message and register it with a unique URI


if __name__ == "__main__":
    mcp.run(transport="stdio")

#main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # List all tools the server provides
            tools_response = await session.list_tools()
            print("Available tools:")
            for tool in tools_response.tools:
                print(f" - {tool.name}: {tool.description}")

            # TODO: List all resources the server provides
            # TODO: Print each resource's URI and description

if __name__ == "__main__":
    asyncio.run(main())
```

## Personalized Prompts for User Interaction

## Complete Client Workflow with MCP